In [1]:
import os
import getpass

from dotenv import load_dotenv
load_dotenv("../keys.env")

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGSMITH_PROJECT"] = "pr-diligent-larch-37"
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [2]:
# 필수 구성 요소 불러오기
from langchain_anthropic import ChatAnthropic
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

## 검색 도구
from langchain_community.tools.tavily_search import TavilySearchResults

## ReAct 기반 에이전트 생성기
from langgraph.prebuilt import create_react_agent

In [5]:
model = ChatAnthropic(model_name="claude-3-sonnet-20240229")

# 대화 상태 기억용 메모리
memory = MemorySaver()

# 검색 툴 (최대 2개 결과 반환)
search = TavilySearchResults(max_results=2)

# 사용할 도구 목록
tools = [search]

# 에이전트 생성
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [ ]:
# Use the agent
config = {"configurable": {"thread_id": "abc123"}}

for step in agent_executor.stream({"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config, stream_mode="values",):
    step["messages"][-1].pretty_print()

In [ ]:
for step in agent_executor.stream({"messages": [HumanMessage(content="whats the weather where I live?")]}, config, stream_mode="values"):
    step["messages"][-1].pretty_print()

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)

for key, value in search_results[0].items():
    print(f"{key}: {value}")

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'title': 'Wednesday, April 2, 2025. San Francisco, CA - Weather Forecast', 'url': 'https://weathershogun.com/weather/usa/ca/san-francisco/480/april/2025-04-02', 'content': 'San Francisco, California Weather: Wednesday, April 2, 2025. Cloudy weather, overcast skies with clouds. Day 55°. Night 50°.', 'score': 0.92731744}, {'title': 'Weather in San Francisco', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1743558404, 'localtime': '2025-04-01 18:46'}, 'current': {'last_updated_epoch': 1743558300, 'last_updated': '2025-04-01 18:45', 'temp_c': 12.8, 'temp_f': 55.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 274, 'wind_dir': 'W', 'pressure_mb': 1013.0, 'pressure_in': 29.92, '

In [9]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("gpt-4o-mini", model_provider="openai")

## 도구를 사용할 수 있는 모델 생성
model_with_tools = model.bind_tools(tools)

In [10]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [12]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Seoul?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Seoul'}, 'id': 'call_5LkPhwgZMBDykmMuHfhjAhgx', 'type': 'tool_call'}]


In [13]:
from langgraph.prebuilt import create_react_agent

model = init_chat_model("gpt-4o-mini", model_provider="openai")
search = TavilySearchResults(max_results=2)
tools = [search]

agent_executor = create_react_agent(model, tools)

In [17]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

hi!
================================== Ai Message ==================================

Hello! How can I assist you today?


In [18]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in seoul?")]}
)

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

whats the weather in seoul?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_9LY17bWni11qQo5e8fCxaD3U)
 Call ID: call_9LY17bWni11qQo5e8fCxaD3U
  Args:
    query: current weather in Seoul
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Weather in Seoul", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Seoul', 'region': '', 'country': 'South Korea', 'lat': 37.5664, 'lon': 126.9997, 'tz_id': 'Asia/Seoul', 'localtime_epoch': 1743562963, 'localtime': '2025-04-02 12:02'}, 'current': {'last_updated_epoch': 1743562800, 'last_updated': '2025-04-02 12:00', 'temp_c': 14.0, 'temp_f': 57.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 5.8, 'wind_kph': 9.4,